# Two-sided Wilcoxon SIGNED test - bias

By: Sam<br>
Updated at: 27/04/23<br>
Compare performance of discretizers <br>

===

Matched pairs settings
- Sample: 540 ML models after discretization (as at 27/04/2023)
- Purpose: pair-wise comparison metrics of the models using different discretization method
    - Test 1.1: Compare bias between pairs of discretizers, regardless of algorithms
    - Test 1.2-1.6: Compare bias between pairs of discretizers, filter by models (CNB, ID3, Knn-Hamming, Knn-VDM, KNN)
    
===

Input data: instrinsic properties and model performance metrics <br>
ChiMerge manuallly and ChiMerge-SB are merged together. <br>
!!! **NB: Please update the data for metrics and export to csv before running this script!

In [1]:
# Import library
import pandas as pd
import numpy as np
from scipy import stats
import math
import random

In [2]:
# Import evaluation data (updated at 27/04/2023)
data = pd.read_csv("all_evaluation_270423.csv")

In [3]:
data.head()

,dataset,disc,param,inconsistency,models,con_features,size,time_disc,accuracy,time_train,bias,variance
0,iris,EWD,4,0.0667,ID3,4,150,0.0164,0.84,0.008698225,0.158,0.055
1,iris,EWD,7,0.02,ID3,4,150,0.0157,0.79,0.010634899,0.158,0.054
2,iris,EWD,10,0.0067,ID3,4,150,0.0164,0.95,0.010643005,0.053,0.014
3,iris,EFD,4,0.04,ID3,4,150,0.0167,0.84,0.009439945,0.158,0.049
4,iris,EFD,7,0.04,ID3,4,150,0.0239,0.95,0.010675907,0.053,0.07


In [4]:
data['disc'].unique() # get list of discretizers

array(['EWD', 'EFD', 'FFD', 'ChiMerge', 'DT', 'ChiMerge-SB'], dtype=object)

In [5]:
data.columns

Index(['dataset', 'disc', 'param', ' inconsistency ', 'models', 'con_features',
       'size', 'time_disc', 'accuracy', 'time_train', 'bias', 'variance'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   dataset          540 non-null    object 
 1   disc             540 non-null    object 
 2   param            540 non-null    int64  
 3    inconsistency   540 non-null    object 
 4   models           540 non-null    object 
 5   con_features     540 non-null    int64  
 6   size             540 non-null    object 
 7   time_disc        540 non-null    float64
 8   accuracy         540 non-null    object 
 9   time_train       540 non-null    object 
 10  bias             540 non-null    object 
 11  variance         540 non-null    object 
dtypes: float64(1), int64(2), object(9)
memory usage: 50.8+ KB


In [7]:
data['models'].unique()

array(['ID3', 'CNB', 'Knn-VDM', 'Knn-Hamming'], dtype=object)

# Wilcoxon_Complete pipeline

## 1.  Wilcoxon signed t test, bias
Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
Implement 4 replications: (DONE)
- Regardless algorithms
- Filter for each algorithm: CNB, ID3, Knn

## 1.1 bias, no filter in algorithm

In [8]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc = pd.to_numeric(data[data['disc']=="EWD"]['bias'],errors='coerce').tolist()
efd_acc = pd.to_numeric(data[data['disc']=="EFD"]['bias'],errors='coerce').tolist()
ffd_acc = pd.to_numeric(data[data['disc']=="FFD"]['bias'],errors='coerce').tolist()
cm_acc = pd.to_numeric(data[(data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")]['bias'],errors='coerce').tolist()
dt_acc = pd.to_numeric(data[data['disc']=="DT"]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc))
print(len(efd_acc))
print(len(ffd_acc))
print(len(cm_acc))
print(len(dt_acc))

# Step 2: filter numeric values
raw_list = [ewd_acc, efd_acc, ffd_acc, cm_acc, dt_acc]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

90
90
120
120
120


In [9]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [10]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [11]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [12]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,1234.5,0.091318
1,ewd vs ffd,1251.0,0.107865
2,ewd vs cm,1163.0,0.060075
3,ewd vs dt,1100.0,0.018986
5,efd vs ffd,1488.5,0.947372
6,efd vs cm,1358.0,0.363353
7,efd vs dt,1283.5,0.200523
10,ffd vs cm,1468.0,0.584134
11,ffd vs dt,1369.5,0.303605
15,cm vs dt,1419.0,0.545064


In [13]:
# Exporting result

model = 'all'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.2 bias, only CNB

In [14]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for CNB
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_cnb = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
efd_acc_cnb = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
ffd_acc_cnb = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
cm_acc_cnb = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
dt_acc_cnb = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="CNB")]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_cnb))
print(len(efd_acc_cnb))
print(len(ffd_acc_cnb))
print(len(cm_acc_cnb))
print(len(dt_acc_cnb))

# Step 2: filter numeric values
raw_list = [ewd_acc_cnb,efd_acc_cnb, ffd_acc_cnb, cm_acc_cnb, dt_acc_cnb]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [15]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [16]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [17]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [18]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,94.0,0.454889
1,ewd vs ffd,84.0,0.176192
2,ewd vs cm,81.5,0.155971
5,efd vs ffd,115.0,0.726184
6,efd vs cm,117.0,0.774529
10,ffd vs cm,105.0,0.502840
11,ffd vs dt,96.5,0.508904
12,cm vs ewd,81.5,0.146517
15,cm vs dt,103.0,0.940481


In [19]:
# Exporting result

model = 'CNB'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.3 bias, only ID3

In [20]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for ID3
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
efd_acc_ID3 = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
ffd_acc_ID3 = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
cm_acc_ID3 = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
dt_acc_ID3 = pd.to_numeric(data[(data['disc']=="DT") & (data['models']=="ID3")]['bias'],errors='coerce').tolist()
# Check number of metrics available for each discretizer
print(len(ewd_acc_ID3))
print(len(efd_acc_ID3))
print(len(ffd_acc_ID3))
print(len(cm_acc_ID3))
print(len(dt_acc_ID3))

# Step 2: filter numeric values
raw_list = [ewd_acc_ID3,efd_acc_ID3, ffd_acc_ID3, cm_acc_ID3, dt_acc_ID3]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
40


In [21]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

# Create loop for Wilcoxon test (two sided) - handle diff = 0
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                              
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

In [22]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [23]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [24]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,81.0,0.016386
1,ewd vs ffd,187.0,0.349333
2,ewd vs cm,207.5,0.607087
3,ewd vs dt,161.0,0.141379
5,efd vs ffd,224.5,0.869297
6,efd vs cm,197.5,0.471568
7,efd vs dt,222.0,0.829013
10,ffd vs cm,100.0,0.032480
11,ffd vs dt,181.0,0.616391
15,cm vs dt,164.0,0.247323


In [25]:
# Exporting result

model = 'ID3'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.4 bias, only KNN-VDM


In [26]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
efd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
ffd_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
cm_acc_knn_vdm = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()
dt_acc_knn_vdm = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-VDM")]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_vdm))
print(len(efd_acc_knn_vdm))
print(len(ffd_acc_knn_vdm))
print(len(cm_acc_knn_vdm))
print(len(dt_acc_knn_vdm))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_vdm,efd_acc_knn_vdm, ffd_acc_knn_vdm, cm_acc_knn_vdm, dt_acc_knn_vdm]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

9
9
12
8
8


In [27]:
print(num_list)

[[0.105, 0.026, 0.053, 0.22, 0.214, 0.171], [0.105, 0.053, 0.053, 0.214, 0.214, 0.214, 0.307, 0.286, 0.29], [0.026, 0.053, 0.132, 0.132, 0.214, 0.197, 0.191, 0.202, 0.32, 0.299, 0.307, 0.147], [0.053, 0.053, 0.198, 0.234, 0.219], [0.053, 0.053, 0.198, 0.234, 0.219]]


In [28]:
print(k)

5


In [29]:
print(test_list)

[[0.171, 0.026, 0.053, 0.22, 0.105], [0.053, 0.214, 0.214, 0.307, 0.105], [0.147, 0.307, 0.132, 0.214, 0.053], [0.053, 0.053, 0.198, 0.234, 0.219], [0.053, 0.053, 0.198, 0.234, 0.219]]


In [30]:
print(len(test_list))

5


In [31]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break  

In [32]:
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
                                
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)

/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3155: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [33]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [34]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [35]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,2.0,0.273322
1,ewd vs ffd,6.0,0.8125
2,ewd vs cm,4.0,0.4375
6,efd vs cm,3.0,0.465209
9,ffd vs efd,6.5,1.0
15,cm vs dt,N/A,N/A


In [36]:
# Exporting result

model = 'KNN-VDM'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.5 bias, only KNN-Hamming

Knn-Hamming is applied for 5 datasets during Capita Selecta phases (post BNAIC)


In [37]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EWD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
efd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="EFD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
ffd_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="FFD") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
cm_acc_knn_hamming = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()
dt_acc_knn_hamming = pd.to_numeric(data[(data['disc']=="ChiMerge") & (data['models']=="Knn-Hamming")]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn_hamming))
print(len(efd_acc_knn_hamming))
print(len(ffd_acc_knn_hamming))
print(len(cm_acc_knn_hamming))
print(len(dt_acc_knn_hamming))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn_hamming,efd_acc_knn_hamming, ffd_acc_knn_hamming, cm_acc_knn_hamming, dt_acc_knn_hamming]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

21
21
28
32
12


In [38]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break 
        
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


In [39]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [40]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)

In [41]:
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,15.0,0.063965
1,ewd vs ffd,30.0,0.518555
2,ewd vs cm,33.0,0.677246
5,efd vs ffd,27.0,0.380371
6,efd vs cm,38.0,0.969727
7,efd vs dt,36.0,0.850098
10,ffd vs cm,26.0,0.339355
11,ffd vs dt,18.0,0.109863
15,cm vs dt,31.5,0.622070
19,dt vs cm,31.5,0.569336


In [42]:
# Exporting result

model = 'KNN-Hamming'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)

## 1.6 bias, KNN (Knn-VDM and Knn-Hamming)

Merge both result from Knn-VDM and KNN-Hamming


In [43]:
# Preparation: Prepare list of metrics for each discretization (test_list)
# Step 1: Obtain bias for each discretization, convert into numeric, string values will be return as NaN
# Filter for Knn-VDM
# Obtain bias for each discretization, convert into numeric, string values will be return as NaN
ewd_acc_knn = pd.to_numeric(data[(data['disc']=="EWD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
efd_acc_knn = pd.to_numeric(data[(data['disc']=="EFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
ffd_acc_knn = pd.to_numeric(data[(data['disc']=="FFD") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
cm_acc_knn = pd.to_numeric(data[((data['disc']=="ChiMerge") | (data['disc'] == "ChiMerge-SB")) & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()
dt_acc_knn = pd.to_numeric(data[(data['disc']=="ChiMerge") & ((data['models']=="Knn-Hamming") | (data['models']=="Knn-VDM"))]['bias'],errors='coerce').tolist()

# Check number of metrics available for each discretizer
print(len(ewd_acc_knn))
print(len(efd_acc_knn))
print(len(ffd_acc_knn))
print(len(cm_acc_knn))
print(len(dt_acc_knn))

# Step 2: filter numeric values
raw_list = [ewd_acc_knn,efd_acc_knn, ffd_acc_knn, cm_acc_knn, dt_acc_knn]
num_list = [] # filter numeric values only
for metric in raw_list:
    metric_new = [x for x in metric if (math.isnan(x) == False)] # using list comprehension
    num_list.append(metric_new)
    
# Step 3: random sample
# For discretization methods with different value, randomly select so that the final sample size are equal
# Reference: https://docs.python.org/3/library/random.html
k = min(len(metric) for metric in num_list)
test_list = []
for metric in num_list:
    random.seed(20)
    if len(metric) > k:
        metric = random.sample(metric, k=k)
    else: metric = metric
    #print(metric)
    test_list.append(metric)

30
30
40
40
20


In [44]:
# Initialiation
disc_key = ['ewd', 'efd', 'ffd', 'cm', 'dt']
disc_value = test_list # list of metrics for each discretization after preparation
test_stat = []
p_value = []
disc_compare = []
# Create dictionary store discretization and series of bias
disc = {}
for key in disc_key:
    for value in disc_value:
        disc[key] = value
        disc_value.remove(value)
        break 
        
# Create loop for Wilcoxon test (two sided)
for i in disc_key:
    for j in disc_key:
        if i != j:
            disc_compare.append(f'{i} vs {j}')
            # print(f'{i} vs {j}')
            # Compute difference list
            diff_list = []
            for m in range(0, len(disc[i])):
                diff = disc[i][m] - disc[j][m]
                diff_list.append(diff)
            if all(item == 0 for item in diff_list) == False: # if the diff list does not contain all 0
                test_stat.append(stats.wilcoxon(diff_list).statistic)
                p_value.append(stats.wilcoxon(diff_list).pvalue)
            else: # if the diff list contain only0, cannot do Wilcoxon test
                error = 'N/A'
                test_stat.append(error)
                p_value.append(error)


/Users/anhtth/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:3141: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [45]:
# Convert 3 lists to dataframe
disc_compare = pd.DataFrame(disc_compare, columns=['disc_compare'])
test_stat = pd.DataFrame(test_stat, columns=['wtest_stat'])
p_value = pd.DataFrame(p_value, columns=['p_value'])

In [46]:
# Result table
wt_result = pd.concat([disc_compare, test_stat, p_value], axis = 1)
# Drop duplicate
wt_result.drop_duplicates(subset=['p_value'], inplace=True)
wt_result

,disc_compare,wtest_stat,p_value
0,ewd vs efd,59.0,0.430679
1,ewd vs ffd,64.5,0.579056
2,ewd vs cm,75.5,0.963226
3,ewd vs dt,57.5,0.403763
5,efd vs ffd,38.5,0.071411
6,efd vs cm,56.0,0.352875
7,efd vs dt,71.0,0.817581
8,ffd vs ewd,64.5,0.611221
9,ffd vs efd,38.5,0.079681
10,ffd vs cm,48.0,0.301054


In [47]:
# Exporting result

model = 'KNN'
metric = 'bias'
test = 'two_sided'

filename = f"wilcoxon_{metric}_{model}-{test}.csv"
wt_result['model'] = model
wt_result['metric'] = metric

wt_result.to_csv(filename, index = False)